In [1]:
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored, brier_score, cumulative_dynamic_auc, as_cumulative_dynamic_auc_scorer
from sksurv.linear_model import CoxPHSurvivalAnalysis

from sksurv.column import encode_categorical
from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM

from sksurv.ensemble import ComponentwiseGradientBoostingSurvivalAnalysis
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

from scipy import stats
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve,f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import numpy as np
import numpy
import pandas as pd

from sklearn.model_selection import ShuffleSplit, GridSearchCV

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC

In [2]:
x_train1 = pd.read_csv('/users/PAS2433/dai417osc/WHI_sp23/data/sp23_nobmd_Xtrain_0820.csv')
y_train1 = pd.read_csv('/users/PAS2433/dai417osc/WHI_sp23/data/sp23_nobmd_Ytrain_competing_risk_0727.csv')
x_test1 = pd.read_csv('/users/PAS2433/dai417osc/WHI_sp23/data/sp23_nobmd_Xtest_0820.csv')
y_test1 = pd.read_csv('/users/PAS2433/dai417osc/WHI_sp23/data/sp23_nobmd_Ytest_competing_risk_0816_10y.csv')

In [3]:
x_train1_grs = x_train1[["AGE","HEIGHTX","WEIGHTX","DIABNW","parental_hip_frac","previous_frac","DRNKSDAY_3_more","CORT","RHEUMAT","Second_Osteo","SCORE",
"RACE_1","RACE_2","RACE_3","RACE_4","RACE_5","SMOKING_2","NUMFALLS_0","NUMFALLS_1","NUMFALLS_2","NUMFALLS_3"]]
x_test1_grs = x_test1[["AGE","HEIGHTX","WEIGHTX","DIABNW","parental_hip_frac","previous_frac","DRNKSDAY_3_more","CORT","RHEUMAT","Second_Osteo","SCORE",
"RACE_1","RACE_2","RACE_3","RACE_4","RACE_5","SMOKING_2","NUMFALLS_0","NUMFALLS_1","NUMFALLS_2","NUMFALLS_3"]]
x_train1_nogrs = x_train1[["AGE","HEIGHTX","WEIGHTX","DIABNW","parental_hip_frac","previous_frac","DRNKSDAY_3_more","CORT","RHEUMAT","Second_Osteo",
"RACE_1","RACE_2","RACE_3","RACE_4","RACE_5","SMOKING_2","NUMFALLS_0","NUMFALLS_1","NUMFALLS_2","NUMFALLS_3"]]
x_test1_nogrs = x_test1[["AGE","HEIGHTX","WEIGHTX","DIABNW","parental_hip_frac","previous_frac","DRNKSDAY_3_more","CORT","RHEUMAT","Second_Osteo",
"RACE_1","RACE_2","RACE_3","RACE_4","RACE_5","SMOKING_2","NUMFALLS_0","NUMFALLS_1","NUMFALLS_2","NUMFALLS_3"]]

In [4]:
# Apply SMOTE to dataset with grs
x_train1smote = pd.concat([x_train1_grs,y_train1[["BKHIPDY"]]], axis=1)
y_train1smote = y_train1[["BKHIP"]]
sm = SMOTE(random_state=2, sampling_strategy="minority")
x_train_ss1, y_train_ss1 = sm.fit_resample(x_train1smote, y_train1smote)

x_train_s1_grs = x_train_ss1.drop("BKHIPDY", axis=1)
y_train_s1_grs = pd.concat([y_train_ss1,x_train_ss1[["BKHIPDY"]]], axis=1)

# Apply SMOTE to dataset without grs
x_train1smote_nogrs = pd.concat([x_train1_nogrs,y_train1[["BKHIPDY"]]], axis=1)
y_train1smote_nogrs = y_train1[["BKHIP"]]
sm = SMOTE(random_state=2, sampling_strategy="minority")
x_train_ss1_nogrs, y_train_ss1 = sm.fit_resample(x_train1smote_nogrs, y_train1smote)

x_train_s1_nogrs = x_train_ss1_nogrs.drop("BKHIPDY", axis=1)
y_train_s1_nogrs = pd.concat([y_train_ss1,x_train_ss1_nogrs[["BKHIPDY"]]], axis=1)

In [5]:
y_train1_array_h = y_train_s1_grs[["BKHIP","BKHIPDY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_train1_array_h]
y_train1_array_rsf_h = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_test1_array_h = y_test1[["BKHIP","BKHIPDY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_test1_array_h]
y_test1_array_rsf_h = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_train1_array_d = y_train1[["Death_10y","DeathDAY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_train1_array_d]
y_train1_array_rsf_d = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_test1_array_d = y_test1[["Death_10y","DeathDAY"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_test1_array_d]
y_test1_array_rsf_d = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_train1_array_cr = y_train1[["BKHIP_cr","BKHIPDY_cr"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_train1_array_cr]
y_train1_array_rsf_cr = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

y_test1_array_cr = y_test1[["BKHIP_cr","BKHIPDY_cr"]].to_numpy()
aux = [(e1,e2) for e1,e2 in y_test1_array_cr]
y_test1_array_rsf_cr = numpy.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

In [6]:
# Model 4 (Bayesian optimization + FRAX CRFs + GRS)
estimator_h = FastSurvivalSVM(alpha=0.5334048246546964, max_iter=1000, optimizer='avltree',
                random_state=0, tol=1e-05)

gcv_h = estimator_h.fit(x_train_s1_grs, y_train1_array_rsf_h)
gcv_d = FastSurvivalSVM().fit(x_train_s1_grs, y_train1_array_rsf_d)

cindex_h = concordance_index_censored(
    y_test1_array_rsf_cr["Status"],
    y_test1_array_rsf_cr["Survival_in_days"],
    gcv_h.predict(x_test1_grs)+gcv_d.predict(x_test1_grs)
)
brier_h = brier_score(
    y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_grs)+gcv_d.predict(x_test1_grs),  
    times=3649
)
auc, mean_auc = cumulative_dynamic_auc(y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_grs)+gcv_d.predict(x_test1_grs),
    times=3649)

print("Model 4: ")
print("C-index: ", cindex_h[0])
print("Brier score: ", brier_h[1])
print("Dynamic mean auc: ", mean_auc)

Model 4: 
C-index:  0.6759072392271182
Brier score:  [0.21056466]
Dynamic mean auc:  0.6846424770853654


In [7]:
# Model 3 (grid search + FRAX CRFs + GRS)
estimator_h = FastSurvivalSVM(alpha=0.5, max_iter=1000, optimizer='avltree',
                random_state=0, tol=1e-05)

gcv_h = estimator_h.fit(x_train_s1_grs,y_train1_array_rsf_h)
gcv_d = FastSurvivalSVM().fit(x_train_s1_grs,y_train1_array_rsf_d)

cindex_h = concordance_index_censored(
    y_test1_array_rsf_cr["Status"],
    y_test1_array_rsf_cr["Survival_in_days"],
    gcv_h.predict(x_test1_grs)+gcv_d.predict(x_test1_grs)
)
brier_h = brier_score(
    y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_grs)+gcv_d.predict(x_test1_grs),  
    times=3649
)
auc, mean_auc = cumulative_dynamic_auc(y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_grs)+gcv_d.predict(x_test1_grs),
    times=3649)

print("Model 3: ")
print("C-index: ", cindex_h[0])
print("Brier score: ", brier_h[1])
print("Dynamic mean auc: ", mean_auc)

Model 3: 
C-index:  0.6406734686250327
Brier score:  [0.11074278]
Dynamic mean auc:  0.6432425925007823


In [8]:
# Model 2 (Bayesian optimization + FRAX CRFs)
estimator_h = FastSurvivalSVM(alpha=0.29175026357002953, max_iter=1000, optimizer='avltree',
                random_state=0, tol=1e-05)


gcv_h = estimator_h.fit(x_train_s1_nogrs, y_train1_array_rsf_h)
gcv_d = FastSurvivalSVM().fit(x_train_s1_nogrs,y_train1_array_rsf_d)

cindex_h = concordance_index_censored(
    y_test1_array_rsf_cr["Status"],
    y_test1_array_rsf_cr["Survival_in_days"],
    gcv_h.predict(x_test1_nogrs)+gcv_d.predict(x_test1_nogrs),
)
brier_h = brier_score(
    y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_nogrs)+gcv_d.predict(x_test1_nogrs),  
    times=3649)
auc, mean_auc = cumulative_dynamic_auc(y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_nogrs)+gcv_d.predict(x_test1_nogrs),
    times=3649)
print("Model 2: ")
print("C-index: ", cindex_h[0])
print("Brier score: ", brier_h[1])
print("Dynamic mean auc: ", mean_auc)

Model 2: 
C-index:  0.6279478267369286
Brier score:  [0.10225828]
Dynamic mean auc:  0.6340296711271866


In [9]:
# Model 1 (grid search + FRAX CRFs)
estimator_h = FastSurvivalSVM(alpha=0.5, max_iter=1000, optimizer='avltree',
                random_state=0, tol=1e-05)

gcv_h = estimator_h.fit(x_train_s1_nogrs,y_train1_array_rsf_h)
gcv_d = FastSurvivalSVM().fit(x_train_s1_nogrs,y_train1_array_rsf_d)

cindex_h = concordance_index_censored(
    y_test1_array_rsf_cr["Status"],
    y_test1_array_rsf_cr["Survival_in_days"],
    gcv_h.predict(x_test1_nogrs)+gcv_d.predict(x_test1_nogrs))
brier_h = brier_score(
    y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_nogrs)+gcv_d.predict(x_test1_nogrs),  
    times=3649)
auc, mean_auc = cumulative_dynamic_auc(y_train1_array_rsf_cr,
    y_test1_array_rsf_cr,
    estimate = gcv_h.predict(x_test1_nogrs)+gcv_d.predict(x_test1_nogrs),
    times=3649)
print("Model 1: ")
print("C-index: ", cindex_h[0])
print("Brier score: ", brier_h[1])
print("Dynamic mean auc: ", mean_auc)

Model 1: 
C-index:  0.6238460963486191
Brier score:  [0.11978375]
Dynamic mean auc:  0.6259956171597859
